In [48]:
import pandas as pd
from tokenizers import SentencePieceBPETokenizer
from datasets import load_dataset

## Data Load

In [49]:
# train = pd.read_csv("./data/ratings_train.txt", sep="\t")
# test = pd.read_csv("./data/ratings_test.txt", sep="\t")

# train = train.dropna(axis=0, subset=["document"])
# test = test.dropna(axis=0, subset=["document"])
# print(train.shape, test.shape)
# train.head()

In [50]:
# train.drop("id", axis=1).to_csv("./data/ratings_train.csv", index=False)
# test.drop("id", axis=1).to_csv("./data/ratings_test.csv", index=False)

## Tokenize with NSMC

In [51]:
# tokenizer = SentencePieceBPETokenizer()
# tokenizer

In [52]:
# train_file = "./data/ratings_train.txt"

# tokenizer.train([train_file], vocab_size=3200)

In [53]:
# for t in train.document.values[:3]:
#     output = tokenizer.encode(t)
#     print(t)
#     print(output.ids)
#     print(output.tokens)
#     print(output.offsets, "\n")

In [54]:
# import os

# if not os.path.isdir("./data/nsmc_sentenctpiece"):
#     os.mkdir("./data/nsmc_sentenctpiece")
# tokenizer.save_model("./data/nsmc_sentenctpiece")

## Data Load

In [55]:
data_files = {"train": "data/ratings_train.csv", "test": "data/ratings_test.csv"}
dataset = load_dataset("csv", data_files= data_files)

Found cached dataset csv (/home/yehoon/.cache/huggingface/datasets/csv/default-a8458de001e53873/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
dataset["train"]

Dataset({
    features: ['document', 'label'],
    num_rows: 149995
})

## Tokenize with LLaMA

In [57]:
from transformers import LlamaTokenizer

In [58]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-13b-hf")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [65]:
def tokenize_function(examples):
#     tokenizer.pad_token = tokenizer.eos_token
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    tokenizer.pad_token_id = 0
#     return tokenizer((examples["document"]), padding="max_length", return_tensors="pt")
    return tokenizer(examples["document"], padding="max_length", truncation=True, max_length=32000, return_tensors="pt")
#     print(examples["document"])
#     return tokenizer(examples["document"])

In [71]:
tokenizer.pad_token_id

In [72]:
dataset["train"]["document"][123132]

'비교적 원작을 잘 담은 영화!'

In [75]:
tokenized_dataset = dataset.map(tokenize_function)

Loading cached processed dataset at /home/yehoon/.cache/huggingface/datasets/csv/default-a8458de001e53873/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-e25f9018438636be.arrow
Loading cached processed dataset at /home/yehoon/.cache/huggingface/datasets/csv/default-a8458de001e53873/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-6b0f9f2703222b77.arrow


In [76]:
# tokenizer.decode(tokenizer.encode(dataset["train"]["document"][123132], bos=True, eos=True))

In [ ]:
len(tokenized_dataset["train"]["input_ids"][0]), tokenized_dataset["train"]["input_ids"]

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(["document"])
tokenized_dataset = tokenized_dataset.remove_columns(["label"])
tokenized_dataset = tokenized_dataset.remove_columns(["attention_mask"])
# tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

In [ ]:
tokenized_dataset.set_format("torch")

In [ ]:
type(tokenized_dataset["train"]["input_ids"]), len(tokenized_dataset["train"]["input_ids"][0])

In [ ]:
tokenizer.encode("[PAD]")

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
train_loader = DataLoader(tokenized_dataset["train"], shuffle=True, batch_size=256)
eval_loader = DataLoader(tokenized_dataset["test"], shuffle=True, batch_size=256)

In [ ]:
len(train_loader.dataset["input_ids"][54354])

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("decapoda-research/llama-7b-hf")

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device="cpu"
model.to(device)

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        print(batch, len(batch["input_ids"][0]))
        batch = {k: v for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
len(train_loader.dataset["attention_mask"][54354]), len(train_loader.dataset["input_ids"][54354])

In [ ]:
model